In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [2]:
%%sql
create table WiderTbl (
    id integer primary key,
    cola integer,
    colb integer,
    colc integer
);

insert into WiderTbl
values (1, 2,3,4),
    (2, 3,NULL,NULL),
    (3, NULL, 2,2),
    (4, NULL, NULL, NULL);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
4 rows affected.


[]

In [ ]:
## やりたいこと
+ 列持ち(wider) -> 行持ち(longer)へ変換する(Rのpivot_longer相当)
+ テーブル設計の鉄則：列持ちのメリットはほとんどない。速攻で行持ちへ変換せよ。

In [6]:
%%sql
drop view if exists LongerTbl;
create view LongerTbl(id, label, value) as (
    select id, 'a', cola
    from WiderTbl
    -- union allは重複を取り除かない
    union all
    select id, 'b', colb
    from WiderTbl
    union all
    select id, 'c', colc
    from WiderTbl
);

select *
from LongerTbl
order by id;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
12 rows affected.


id,label,value
1,a,2
1,b,3
1,c,4
2,c,None
2,a,3
2,b,None
3,b,2
3,c,2
3,a,None
4,a,None
